# Practice GAN usgin tensorflow

### Setup

In [2]:
from absl import logging

import imageio
import PIL.Image
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
tf.random.set_seed(0)

import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import time

try:
    from google.colab import files
except ImportError:
    pass

from IPython import display
from skimage import transform

### Define helper function

In [3]:
""" We could retrieve this value from module.get_input_shapes() if we didn't know
    beforehand which module we will be using"""

latent_dim = 512

"""
Interpolates between two vectors that are non-zero and don't both lie on a 
line going through origin. First normalizes v2 to have the same norm as v1.
Then interpolates between the tow vectors on the hypersphere.
"""

def interpolate_hypersphere(v1, v2, num_steps):
    v1_norm = tf.norm(v1)
    v2_norm = tf.norm(v2)
    v2_normalzied = v2 * (v1_norm / v2_norm)

    vectors = []
    for step in range(num_steps):
        interpolated = v1 + (v2_normalzied - v1) * step / (num_steps - 1)
        interpolated_norm = tf.norm(interpolated)
        interpolated_normalized = interpolated * (v1_norm / interpolated_norm)
        vectors.append(interpolated_normalized)
    return tf.stack(vectors)

# simple way to display an image
def display_image(image):
    image = tf.constant(image)
    image = tf.image.convert_image_dtype(image, tf.uint8)
    return PIL.Image.fromarray(image.numpy())

# Given a set of images, show an animation
def animate(iamges):
    images = np.array(images)
    converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
    imageio.mimsave('./animation.gif', converted_images)
    return embed.embed_file('./animation.gif')

logging.set_verbosity(logging.ERROR)


In [4]:
# load GAN model
progan = hub.load("https://tfhub.dev/google/progan-128/1").signatures['default']

progan


<ConcreteFunction pruned(latent_vector) at 0x26BAD737FA0>

In [ ]:
def interpolate_between_vectors():
    v1 = tf.random.normal([latent_dim])
    v2 = tf.random.normal([latent_dim])

    # Creates a tensor with 25 steps of interpolation between v1 and v2.
    vectors = interpolate_hypersphere(v1, v2, 50)

    # Uses module to generate images from the latent space.
    interpolated_images = progan(vectors)['default']

    return interpolated_images

interpolated_images = interpolate_between_vectors()
animate(interpolated_images)